<a href="https://colab.research.google.com/github/tiagolobato/RioGrandeSulRotas/blob/main/Rotas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Instalar a SDK

In [ ]:
!pip install -q -U google-generativeai

## Configurar a API KEY

In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY='YOUR_API_KEY'
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import requests
import pdb
import re

from bs4 import BeautifulSoup

def get_cities_list():

  def extract_card_data(card_div):
    """
    Extrai informações relevantes de um único elemento de cartão (div).

    Args:
        card_div: O elemento BeautifulSoup que representa a div do cartão.

    Returns:
        Um dicionário contendo as informações extraídas do cartão, ou um dicionário vazio se houver algum erro.
    """
    data = {}
    try:
      header = card_div.find('div', class_='panel-heading card-header card-item-title')
      data['titulo'] = header.text.strip() if header else None

      body = card_div.find('div', class_='panel-body card-body card-item-content')
      #pdb.set_trace()
      if body:
        # Extrair informações usando expressões regulares pode ser mais robusto,
        # mas aqui usamos uma abordagem simples com base na estrutura HTML observada.
        strong_tags = body.find_all('strong')
        data['origem'] = strong_tags[0].next_sibling.strip()
        data['destino'] = strong_tags[1].next_sibling.strip()
        data['atualizacao'] = strong_tags[2].next_sibling.strip()
        data['descricao'] = strong_tags[3].next_sibling.strip()
        # Obter link do mapa (se existir)
        links_with_onclick = body.find_all('a', href="javascript:void(0);", onclick=True)
        map_links= []

        for link in links_with_onclick:
          onclick_value = link['onclick']
          match = re.search(r"window\.open\('(.*?)'", onclick_value)
          if match:
            map_link = match.group(1)
            map_links.append(map_link)

        map_link = body.find('a', class_='btn btn-success w-100')
        data['rota1'] = map_links[0] if len(map_links) > 0 else None
        data['rota2'] = map_links[1] if len(map_links) > 1 else None
    except (AttributeError, IndexError) as e:
      print(f"Erro ao extrair dados do cartão: {e}")

    return data

  def get_card_list_data(url):
    """
    Busca a lista de cartões da URL fornecida e extrai as informações de cada cartão.

    Args:
        url: A URL da página a ser analisada.

    Returns:
        Uma lista de dicionários, onde cada dicionário contém as informações de um cartão.
        Retorna uma lista vazia se nenhum cartão for encontrado ou se houver um erro.
    """
    try:
      response = requests.get(url)
      response.raise_for_status()

      soup = BeautifulSoup(response.content, 'html.parser')
      card_divs = soup.find_all('div', class_='panel card panel-default card-item')

      card_data_list = []
      for card_div in card_divs:
        card_data = extract_card_data(card_div)
        card_data_list.append(card_data)

      return card_data_list
    except requests.exceptions.RequestException as e:
      print(f"Erro na solicitação: {e}")
      return []

  url = "https://crbm.app.br/gestao-de-rotas/engine.php?class=RotaCardList"
  card_data_list = get_card_list_data(url)
  return card_data_list

## Initialize the Generative Model

Before you can make any API calls, you need to initialize the Generative Model.

In [ ]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""

import google.generativeai as genai


# Set up the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]



## Generate text

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)
cities= get_cities_list()
convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": [f"{cities}"]
  },
  {
    "role": "model",
    "parts":["ok"]
  },
  {
    "role": "user",
    "parts": ["Voce deve responder seguindo o modelo abaixo, e somente o modelo abaixo, não responda nada alem disso, caso pergutem algo que não seja relacionado as rotas das cidades você deve responder 'Desculpe, tenho conhecimento apenas de rotas entre cidades, caso queira fazer perguntas genericas visite o https://gemini.google.com/'. Se voce não achar a cidades que foram perguntandas voce deve responder 'Não encontrei o trajeto das cidades, conferir no site: https://crbm.app.br/gestao-de-rotas/index.php?class=RotaCardList'.Caso não tenha a rota 2 deve ignorar esse campo, Segue exemplo- Pergunta: Quero ir de VACARIA para RIO GRANDE\n\nResposta:\nTitulo: VACARIA x RIO GRANDE\nDescrição: OPÇÃO 1 - VACARIA A RIO GRANDE VIA SÃO JOSE DO NORTE - OPÇÃO 2 - VACARIA A RIO GRANDE VIA ALEGRETE\nUltima atualização: 07/05/2024 17:44\nRota 1:https://www.google.com/maps/d/u/0/viewer?usp=sharing&mid=1iJuIEOo6P_YC-2-DbkTWD7bFv8L8C90\nRota 2: https://www.google.com/maps/d/viewer?mid=11WP2Ubom3zuqpcjhoImHwHkaGDYuAJI"]
  },
  {
    "role": "model",
    "parts":["ok"]
  },
])


print("Olá")
prompt = input("Digite a cidade de origem e a cidade de destino: ")
while prompt != "sair":
  convo.send_message(f"Pergunta: {prompt}")
  print(convo.last.text)
  prompt = input("Digite a cidade de origem e a cidade de destino: ")



